# Timely and Effective Care

We're entering an awesome era where open source and open datasets allow many more people to tackle life's challenges, including healthcare. What once required teams of researchers can now be done by a few people with coffee and laptops. 

The Center for Medicare and Medicaid Services (CMS) has made many datasets available, including a data set tracking hospitals' ability to provide timely and effective care across various use cases. 

Timely and Effective care is measured by the percent of patients who meet certain quality metrics, such as whether or not patients got key drugs within 30 minutes of their hospital visit. These are scored by "measures", and you can get detailed descriptions of those measures here: [Timely and Effective Care](https://www.medicare.gov/hospitalcompare/about/timely-effective-care.html)

This study seeks to explore what characteristics lead to timely and effective care. Health systems can potentially use this information to make smarter investments in quality and safety.

This work will be done in three phases:
- Preprocessing & EDA (current)
- Analysis and Predictive Modeling
- Presentation & Publication (interactive charting)

### Note before attempting to use this notebook

Make sure you have the libraries you see below installed! Additionally, this pulls from Medicare's Data API. You'll need to use [this page](https://dev.socrata.com/foundry/data.medicare.gov/3z8n-wcgr) to obtain access to the data without throttling limits. However, the present code is just fine for this project, as it doesn't presently depend on fast or repeat access to the data. 

In [1]:
#Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sodapy import Socrata
import re

import pandas_bokeh
pandas_bokeh.output_notebook()

plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 500)

In [2]:
#Function to import Medicare data tables

def medicare_importer(token, password, user_name, medicare_data, data_code, limit):
    '''Importer for Medicare Data API. Takes in token, password, user_name, medicare_data, data_code, and limit
       to return a DataFrame for a given table.
       
       Socrata is the mechanism that returns data. Make sure to import Socrata before running this funtion.
       
       token(str) = your app token
       password(str) = your app password
       user_name(str) = your user name
       medicare_data(str) = likely data.medicare.gov but may vary
       data_code(str) = the code for a specific table. For instance, Timely & Effective Care is 3z8n-wcgr.
       limit(int) = the maximum number of rows you would like to grab. Defaults to 1000, so adjust upwards for complete data sets.
       '''
    
    #call Socrata to connect to Medicare data
    client = Socrata(medicare_data,
                     token,
                     username=user_name,
                     password=password)
    
    return pd.DataFrame(client.get(data_code, limit=limit)) #return relevant dataframe

In [3]:
#Import data for timely & effective care

app_token = ''
app_password = ''
user_name = ''
medicare_data = "data.medicare.gov"
timely_effective_care = "3z8n-wcgr"

te_data = medicare_importer(app_token, app_password, user_name, medicare_data, timely_effective_care, 120000)

#reorder columns
te_data = te_data[['address', 'city', 'state', 'zip_code', 'county_name', 'hospital_name', 'provider_id', 'location', 
                   'location_address', 'location_city', 'location_state', 'condition','footnote', 'measure_start_date', 
                   'measure_end_date', 'measure_id', 'measure_name', 'phone_number',  'sample', 'score']] 
te_data.head()

,address,city,condition,county_name,footnote,hospital_name,location,location_address,location_city,location_state,measure_end_date,measure_id,measure_name,measure_start_date,phone_number,provider_id,sample,score,state,zip_code
0,595 WEST CAROLINA AVENUE,VARNVILLE,Emergency Department,HAMPTON,NaN,HAMPTON REGIONAL MEDICAL CENTER,"{'type': 'Point', 'coordinates': [-81.087692, 32.855258]}",595 WEST CAROLINA AVENUE,VARNVILLE,SC,2016-12-31T00:00:00.000,OP_22,Left before being seen,2016-01-01T00:00:00.000,8039432771,420072,13179,4,SC,29944
1,3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,Preventive Care,ORANGEBURG,2 - Data submitted were based on a sample of cases/patients.,TRMC OF ORANGEBURG & CALHOUN,"{'type': 'Point', 'coordinates': [-80.832814, 33.540783]}",3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,2017-03-31T00:00:00.000,IMM_2,Immunization for influenza,2016-10-01T00:00:00.000,8035332460,420068,545,91,SC,29115
2,300 SINGLETON RIDGE ROAD,CONWAY,Colonoscopy care,HORRY,NaN,CONWAY MEDICAL CENTER,"{'type': 'Point', 'coordinates': [-79.00287, 33.786517]}",300 SINGLETON RIDGE ROAD,CONWAY,SC,2016-12-31T00:00:00.000,OP_29,Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients,2016-01-01T00:00:00.000,8433478037,420049,64,78,SC,29526
3,2095 HENRY TECKLENBURG DRIVE,CHARLESTON,Preventive Care,CHARLESTON,2 - Data submitted were based on a sample of cases/patients.,BON SECOURS-ST FRANCIS XAVIER HOSPITAL,"{'type': 'Point', 'coordinates': [-80.042264, 32.810751]}",2095 HENRY TECKLENBURG DRIVE,CHARLESTON,SC,2017-03-31T00:00:00.000,IMM_2,Immunization for influenza,2016-10-01T00:00:00.000,8434021006,420065,547,88,SC,29414
4,3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,Cancer care,ORANGEBURG,NaN,TRMC OF ORANGEBURG & CALHOUN,"{'type': 'Point', 'coordinates': [-80.832814, 33.540783]}",3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,2016-12-31T00:00:00.000,OP_33,External Beam Radiotherapy for Bone Metastases,2016-01-01T00:00:00.000,8035332460,420068,29,72,SC,29115


In [4]:
te_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110239 entries, 0 to 110238
Data columns (total 20 columns):
address               110239 non-null object
city                  110239 non-null object
condition             110239 non-null object
county_name           109894 non-null object
footnote              67791 non-null object
hospital_name         110239 non-null object
location              101522 non-null object
location_address      110239 non-null object
location_city         110239 non-null object
location_state        110239 non-null object
measure_end_date      110239 non-null object
measure_id            110239 non-null object
measure_name          110239 non-null object
measure_start_date    110239 non-null object
phone_number          110239 non-null object
provider_id           110239 non-null object
sample                105446 non-null object
score                 110239 non-null object
state                 110239 non-null object
zip_code              110239 non-nu

As it was sent, this data set has no numeric datatypes. We would expect the score to be a numeric type. Let's explore what prevented this from happening. 

In [5]:
te_data.score.unique() #Interesting what gets included in score. There is a high variance of numbers in the scoring here because each score represents a different quality measure, and each quality measure is different

array(['4', '91', '78', '88', '72', '100', '98', '99', '94', '0',
       'Low (0 - 19,999 patients annually)', '93', '1', '75', '9', '2',
       'Not Available', 'High (40,000 - 59,999 patients annually)', '73',
       '97', '76', '3', 'Medium (20,000 - 39,999 patients annually)',
       '96', '86', '85', '50', '79', '92', '61', '5', '55', '80', '70',
       '84', '95', '58', '82', '90', '63', '65', '89', '83', '64', '6',
       '7', '25', '8', '47', '71', '87', '77', '74', '57', '53', '54',
       '81', '69', '35', '67', '31', '12', '62', '68', '59', '39', '56',
       '66', '60', '48', '40', '10', '36', '27', '22', '49', '13', '19',
       '14', '52', '43', '37', '51', '20', '44', '46', '227', '185',
       '119', '11', '30', '32', '23', '45', '17', '42', '26', '15', '28',
       '38', '21', '41', '24', '34', '29', '18', '33', '16', '369', '123',
       '125', '113', '229', '199', '288', '130', '318', '241', '251',
       '305', '155', '273', '232', '189', '139', '138', '167', '106',

In [6]:
import re #needed for deeper text analysis

letter_finder = r'[A-Za-z]' #regex pattern to identify non-numeric values in strings
has_letters = te_data.score.str.contains(letter_finder) #filter using that pattern
te_data.loc[has_letters].score.value_counts() #Counts of these non-numeric values

Not Available                                 52164
Low (0 - 19,999 patients annually)             1303
Medium (20,000 - 39,999 patients annually)      962
Very High 60,000+ patients annually             699
High (40,000 - 59,999 patients annually)        603
Name: score, dtype: int64

In [7]:
te_data.score.str.contains(letter_finder).value_counts() 

True     55731
False    54508
Name: score, dtype: int64

Based on the observations above, we learn two things:
- More than 50% of the data is non-numeric
- Close to 50% of scores are not available

The impact here will vary depending on the measure we're studying. 

We want to figure out characteristics of these data to determine what to do with 
1. Missing values  
2. Categorical values

## Missing Values

How many values are missing by state?

In [8]:
score_not_available = te_data.score.str.contains('Not') #filter to the not available values

((te_data.loc[score_not_available].groupby('state')['score'].count()\
  /te_data.groupby('state')['score'].count())*100).sort_values(ascending=False)

state
MP    100.000000
AS    100.000000
GU     89.130435
PR     88.879599
VI     82.608696
SD     76.461769
MT     70.967742
ND     69.861660
AK     67.984190
ID     66.666667
KS     65.632992
VT     62.111801
WY     61.180124
IA     59.220390
NE     58.768930
HI     58.223062
MS     57.665904
LA     55.535258
OK     54.436197
CO     53.532609
TX     52.056979
NM     51.537646
NV     49.316770
WA     48.212560
MO     48.136646
AZ     47.881828
MN     47.424749
AR     46.840580
UT     46.691871
CA     44.995537
WV     44.897959
WI     44.720497
NH     43.478261
KY     43.048256
IL     42.560386
OH     42.404092
OR     41.304348
ME     41.238472
IN     40.289855
MI     40.092931
PA     39.715230
GA     39.690382
AL     39.082656
DE     38.509317
FL     38.445180
DC     38.043478
TN     37.962963
MD     37.622005
NY     36.649616
SC     36.304348
CT     35.203366
MA     35.127674
NC     34.989648
VA     34.833760
NJ     31.818182
RI     31.620553
Name: score, dtype: float64

The greatest incident of "Not available" is in non-US states. Let's see if there's anything interesting of note by measure. 

In [9]:
#Here are the measures and their values, notice a trend?
total_measures = te_data.measure_name.value_counts()
score_available_measures = te_data.loc[(score_not_available == False), 'measure_name'].value_counts()
score_available_measures_percent = ((score_available_measures/total_measures) * 100)

measures_records = pd.DataFrame({'available_records': score_available_measures, 
                                 'total': total_measures, 
                                 'percent_available': score_available_measures_percent}).sort_values(by='percent_available', ascending=False)
measures_records

,available_records,total,percent_available
Healthcare workers given influenza vaccination,4257,4793,88.817025
Immunization for influenza,4154,4793,86.668058
ED1,3919,4793,81.765074
ED2,3886,4793,81.076570
Door to diagnostic eval,3816,4793,79.616107
OP 18,3812,4793,79.532652
Median time to pain med,3602,4793,75.151262
Emergency department volume,3567,4793,74.421031
Left before being seen,3560,4793,74.274984
Appropriate care for severe sepsis and septic shock,3070,4793,64.051742


# The Measures We Should Measure

In [10]:
#Find needed number of samples for a given measure
corchrans_formula = ((1.96**2)*.5*.5)/(.05**2) #filter to ensure we're grabbing appropriate sample size
print('For a 95% confidence, Corchran\'s formula tells us we need {} samples, so we\'ll keep only rows where we have sufficient data.'.format(corchrans_formula))

For a 95% confidence, Corchran's formula tells us we need 384.1599999999999 samples, so we'll keep only rows where we have sufficient data.


In [11]:
#This creates a dataframe indicating which measures we should measure
measures_records_use = measures_records.loc[measures_records.available_records > corchrans_formula].reset_index(drop=False).rename(columns={'index': 'measure'})
measures_records_use

,measure,available_records,total,percent_available
0,Healthcare workers given influenza vaccination,4257,4793,88.817025
1,Immunization for influenza,4154,4793,86.668058
2,ED1,3919,4793,81.765074
3,ED2,3886,4793,81.076570
4,Door to diagnostic eval,3816,4793,79.616107
5,OP 18,3812,4793,79.532652
6,Median time to pain med,3602,4793,75.151262
7,Emergency department volume,3567,4793,74.421031
8,Left before being seen,3560,4793,74.274984
9,Appropriate care for severe sepsis and septic shock,3070,4793,64.051742


In [13]:
te_data.head()

,address,city,state,zip_code,county_name,hospital_name,provider_id,location,location_address,location_city,location_state,condition,footnote,measure_start_date,measure_end_date,measure_id,measure_name,phone_number,sample,score
0,595 WEST CAROLINA AVENUE,VARNVILLE,SC,29944,HAMPTON,HAMPTON REGIONAL MEDICAL CENTER,420072,"{'type': 'Point', 'coordinates': [-81.087692, 32.855258]}",595 WEST CAROLINA AVENUE,VARNVILLE,SC,Emergency Department,NaN,2016-01-01T00:00:00.000,2016-12-31T00:00:00.000,OP_22,Left before being seen,8039432771,13179,4
1,3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,29115,ORANGEBURG,TRMC OF ORANGEBURG & CALHOUN,420068,"{'type': 'Point', 'coordinates': [-80.832814, 33.540783]}",3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,Preventive Care,2 - Data submitted were based on a sample of cases/patients.,2016-10-01T00:00:00.000,2017-03-31T00:00:00.000,IMM_2,Immunization for influenza,8035332460,545,91
2,300 SINGLETON RIDGE ROAD,CONWAY,SC,29526,HORRY,CONWAY MEDICAL CENTER,420049,"{'type': 'Point', 'coordinates': [-79.00287, 33.786517]}",300 SINGLETON RIDGE ROAD,CONWAY,SC,Colonoscopy care,NaN,2016-01-01T00:00:00.000,2016-12-31T00:00:00.000,OP_29,Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients,8433478037,64,78
3,2095 HENRY TECKLENBURG DRIVE,CHARLESTON,SC,29414,CHARLESTON,BON SECOURS-ST FRANCIS XAVIER HOSPITAL,420065,"{'type': 'Point', 'coordinates': [-80.042264, 32.810751]}",2095 HENRY TECKLENBURG DRIVE,CHARLESTON,SC,Preventive Care,2 - Data submitted were based on a sample of cases/patients.,2016-10-01T00:00:00.000,2017-03-31T00:00:00.000,IMM_2,Immunization for influenza,8434021006,547,88
4,3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,29115,ORANGEBURG,TRMC OF ORANGEBURG & CALHOUN,420068,"{'type': 'Point', 'coordinates': [-80.832814, 33.540783]}",3000 ST MATTHEWS RD BOX 1806,ORANGEBURG,SC,Cancer care,NaN,2016-01-01T00:00:00.000,2016-12-31T00:00:00.000,OP_33,External Beam Radiotherapy for Bone Metastases,8035332460,29,72


We've developed a filter that narrows our research to the applicable cases and determined which quality measures we're able to research. 

The next step, "Categorical Values", will help us better understand what scores actually are and how to deal with the small number that were in categories rather than buckets. 

# Study 1: Healthcare Workers Given Influenza Vaccination

From the Preventive Care portion of [Timely and Effective Care](https://www.medicare.gov/hospitalcompare/About/Timely-Effective-Care.html)

- Study began October 2017 and concluded March 2018.
- Influenza, or the "flu," is a respiratory illness that is caused by flu viruses and easily spread from person to person.
- There are over 200,000 hospitalizations from the flu on average every year. An average of 36,000 Americans die annually due to the flu and its complications.
- Hospital staff and healthcare workers who are infected with the flu virus can transmit the virus to coworkers and patients, including those at higher risk for getting very sick from the flu. To reduce the spread of flu within a hospital, the Centers for Disease Control and Prevention recommends that all healthcare workers who work in a healthcare setting get the flu vaccine (“flu shot”) each year.
- Vaccinating healthcare workers has been found to reduce the risk of flu illness, medical visits, antibiotic use, and flu-related deaths. It is recommended that all healthcare facilities provide the flu vaccine to their healthcare workers. This measure shows the percentage of all healthcare workers in a hospital that received the flu vaccine.

- Higher percentages are better.

In [55]:
#We're taking on some basic steps to get the data we need and preprocess it. 

def study(df, measure_name, available_only = True):
    '''Takes in Timely & Effective Care data and a string for measure name to return the desired study. 
       A user can specify whether to keep or eliminate rows with missing scores'''
    
    measure_filter = df['measure_name'] == measure_name
    available_filter = (df.score != 'Not Available')
    
    total_length = len(df.loc[measure_filter])
    missing_length = len(df.loc[(available_filter == False) & measure_filter])
    
    
    if available_only == True:
        
        print('** {} out of {} were missing scores. These were removed. You can keep them by setting "available_only" to "False" **\n'.format(missing_length, total_length))        
        return df.loc[(measure_filter) & (available_filter)]
    
    else:
        
        print('{} out of {} are missing scores.'.format(missing_length, total_length))
        return df.loc[(measure_filter)]    

hw_influenza_filter = study(te_data, 'Healthcare workers given influenza vaccination')
hw_influenza_study.score = pd.to_numeric(hw_influenza_study.score) #ensure that the scores are numeric
hw_influenza_study['sample'] = pd.to_numeric(hw_influenza_study['sample']) #ensure that the sample sizes are numeric

hw_influenza_study.info() #Null scores are eliminated and score is now a numeric target

** 536 out of 4793 were missing scores. These were removed. You can keep them by setting "available_only" to "False" **

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4257 entries, 2971 to 110231
Data columns (total 20 columns):
address               4257 non-null object
city                  4257 non-null object
state                 4257 non-null object
zip_code              4257 non-null object
county_name           4249 non-null object
hospital_name         4257 non-null object
provider_id           4257 non-null object
location              3987 non-null object
location_address      4257 non-null object
location_city         4257 non-null object
location_state        4257 non-null object
condition             4257 non-null object
footnote              0 non-null object
measure_start_date    4257 non-null object
measure_end_date      4257 non-null object
measure_id            4257 non-null object
measure_name          4257 non-null object
phone_number          4257 non-null obj

In [57]:
hw_influenza_study.head()

,address,city,state,zip_code,county_name,hospital_name,provider_id,location,location_address,location_city,location_state,condition,footnote,measure_start_date,measure_end_date,measure_id,measure_name,phone_number,sample,score
2971,1208 6TH AVE E,SUPERIOR,MT,59872,MINERAL,MINERAL COMMUNITY HOSPITAL,271331,"{'type': 'Point', 'coordinates': [-114.876579, 47.184726]}",1208 6TH AVE E,SUPERIOR,MT,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,4068224841,69,91
25801,1101 OCILLA ROAD,DOUGLAS,GA,31533,COFFEE,COFFEE REGIONAL MEDICAL CENTER,110089,"{'type': 'Point', 'coordinates': [-82.86346, 31.509589]}",1101 OCILLA ROAD,DOUGLAS,GA,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,9123835620,1163,99
25868,1500 N RITTER AVE,INDIANAPOLIS,IN,46219,MARION,COMMUNITY HOSPITAL EAST,150074,"{'type': 'Point', 'coordinates': [-86.073788, 39.786457]}",1500 N RITTER AVE,INDIANAPOLIS,IN,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,3173555411,5085,96
25879,40 1ST STREET SE,WAUKON,IA,52172,ALLAMAKEE,VETERANS MEMORIAL HOSPITAL,161318,"{'type': 'Point', 'coordinates': [-91.474446, 43.268092]}",40 1ST STREET SE,WAUKON,IA,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,5635683411,241,93
25903,400 S CLARK ST,BUTTE,MT,59701,SILVER BOW,ST JAMES HEALTHCARE,270017,"{'type': 'Point', 'coordinates': [-112.546068, 46.00928]}",400 S CLARK ST,BUTTE,MT,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,4067232500,864,95


In [58]:
#Great dataset to work with. Let's better understand it with summary statistics

hw_influenza_study.describe() #You can start getting a sense for the distribution here. It's interesting that many practices got 0 scores.

,sample,score
count,4257.000000,4257.000000
mean,1961.540756,87.158797
std,2849.255106,13.315368
min,16.000000,0.000000
25%,359.000000,82.000000
50%,958.000000,91.000000
75%,2409.000000,97.000000
max,39112.000000,100.000000


More than half of these organizations score above 90%, but it's quite curious the ones that don't. 

In [107]:
hw_influenza_study['score'].plot_bokeh(kind='hist',
                                       bins = 100,
                                       figsize=(800, 450),
                                       xticks = np.arange(0,101,10),
                                       xlabel = 'Percent of Workers Receiving Vaccine',
                                       ylabel = 'Number of Hospitals');

Is the significant number of hospitals whose vaccination rates are below 90% a cause for alarm? Let's figure it out using published math models.

- [Basic Reproduction Numbers](https://en.wikipedia.org/wiki/Basic_reproduction_number) - The number of cases of a given illness one is expected to generate while infected
- [CDC Guide to Vaccination Effectiveness](https://www.cdc.gov/flu/professionals/vaccination/effectiveness-studies.htm) - Vaccine effectiveness for influenza was 40% in 2017-2018
- [Thoughtscapism Blog's Simple Math of Herd Immunity](https://thoughtscapism.com/2015/04/20/the-simple-math-of-herd-immunity/) - Formula used below

In [60]:
def vaccine_coverage(reproduction, vaccine_effectiveness):
    '''Calculates percent of population requiring vaccines by taking in the number of cases an infected individual is expected to generate
       and the effectiveness of a given vaccine'''
    
    herd_immunity_threshold = (1-(1/reproduction))
    vaccine_coverage = herd_immunity_threshold/vaccine_effectiveness
    
    return vaccine_coverage

influenza_reproduction_rate = 2.5
influenza_vaccine_effectiveness = .4

print('Percent of vaccinations needed: ' + str(vaccine_coverage(influenza_reproduction_rate, influenza_vaccine_effectiveness)*100) + '%')

Percent of vaccinations needed: 149.99999999999997%


As we can see, it's impossible to hit the herd immunity threshold against influenza, but to reduce hospital acquired infections, healthcare workers should vaccinate at far higher rates than a [general population who is doubtful of the need for vaccinations](https://www.npr.org/sections/health-shots/2015/11/27/456202280/many-americans-believe-they-dont-need-the-flu-vaccine) (NPR).

Typical curiosities at this point:
- What factors caused this number?
- What is the impact of improvements on this number?
- How can we improve this number?

In [88]:
hw_influenza_study.loc[hw_influenza_study.city == 'LITTLE ROCK']

,address,city,state,zip_code,county_name,hospital_name,provider_id,location,location_address,location_city,location_state,condition,footnote,measure_start_date,measure_end_date,measure_id,measure_name,phone_number,sample,score
36160,TWO ST VINCENT CIRCLE,LITTLE ROCK,AR,72205,PULASKI,CHI-ST VINCENT INFIRMARY,040007,NaN,TWO ST VINCENT CIRCLE,LITTLE ROCK,AR,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,5015523000,2883,90
57711,1701 S SHACKLEFORD ROAD,LITTLE ROCK,AR,72211,PULASKI,"ARKANSAS HEART HOSPITAL, LLC",040134,"{'type': 'Point', 'coordinates': [-92.394421, 34.739159]}",1701 S SHACKLEFORD ROAD,LITTLE ROCK,AR,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,5012197000,1190,75
69079,"9601 INTERSTATE 630, EXIT 7",LITTLE ROCK,AR,72205,PULASKI,BAPTIST HEALTH MEDICAL CENTER-LITTLE ROCK,040114,"{'type': 'Point', 'coordinates': [-92.4007, 34.75356]}","9601 INTERSTATE 630, EXIT 7",LITTLE ROCK,AR,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,5012022000,4807,75
88670,4301 WEST MARKHAM STREET MAIL SLOT 612,LITTLE ROCK,AR,72205,PULASKI,UAMS MEDICAL CENTER,040016,"{'type': 'Point', 'coordinates': [-92.320998, 34.751583]}",4301 WEST MARKHAM STREET MAIL SLOT 612,LITTLE ROCK,AR,Preventive Care,NaN,2017-10-01T00:00:00.000,2018-03-31T00:00:00.000,IMM_3_OP_27_FAC_ADHPCT,Healthcare workers given influenza vaccination,5016865000,15013,91


Loading BokehJS ...

In [99]:
np.arange(0,101,10)

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100])